In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.losses import Loss
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train_base_dir = "/kaggle/input/isic-2019-challenge/ISIC_2019_Training_Input/ISIC_2019_Training_Input"
test_base_dir = "/kaggle/input/isic-2019-challenge/ISIC_2019_Test_Input/ISIC_2019_Test_Input"
train_groundtruth_path = "/kaggle/input/isic-2019-challenge/ISIC_2019_Training_GroundTruth.csv"
test_groundtruth_path = "/kaggle/input/isic-2019-test-ground-truth/ISIC_2019_Test_GroundTruth.csv"
labels = ['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC']
img_size = 75
batch_size = 64
epochs = 50

In [ ]:
train_groundtruth = pd.read_csv(train_groundtruth_path)
train_groundtruth['label'] = train_groundtruth[labels].idxmax(axis=1)

test_groundtruth = pd.read_csv(test_groundtruth_path)
test_groundtruth['label'] = test_groundtruth[labels].idxmax(axis=1)

In [ ]:
def preprocess_image(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [img_size, img_size])

    label = tf.one_hot(label, len(labels))
    return img, label

In [ ]:
def normalize_image(img, label):
    img = tf.cast(img, tf.float32) / 255.0
    return img, label

In [ ]:
def augment_image(img, label):

    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_flip_up_down(img)

    k = tf.random.uniform([], minval=0, maxval=4, dtype=tf.int32)
    img = tf.image.rot90(img, k)

    crop_frac = tf.random.uniform([], 0.8, 1.0)
    crop_size = tf.cast(crop_frac * tf.cast(tf.shape(img)[0:2], tf.float32), tf.int32)
    img = tf.image.random_crop(img, size=tf.concat([crop_size, [3]], axis=0))
    img = tf.image.resize(img, [img_size, img_size])


    img = tf.image.random_brightness(img, max_delta=0.1)
    img = tf.image.random_contrast(img, lower=0.8, upper=1.2)

    return img, label


In [ ]:

file_paths = [os.path.join(train_base_dir, row['image'] + '.jpg') for _, row in train_groundtruth.iterrows()]
labels_indices = [labels.index(row['label']) for _, row in train_groundtruth.iterrows()]

train_paths, val_paths, train_labels, val_labels = train_test_split(
    file_paths, labels_indices, test_size=0.2, random_state=42
)

test_paths = [os.path.join(test_base_dir, row['image'] + '.jpg') for _, row in test_groundtruth.iterrows()]
test_labels = [labels.index(row['label']) for _, row in test_groundtruth.iterrows()]

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
train_dataset = train_dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.map(normalize_image, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.map(augment_image, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))
val_dataset = val_dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.map(normalize_image, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_paths, test_labels))
test_dataset = test_dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(normalize_image, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

### ResNet152V2

In [ ]:
base_model=tf.keras.applications.ResNet152V2(
    include_top=False,
    weights="imagenet",
    input_shape=(75,75,3)
)


for layer in base_model.layers[:300]:
    layer.trainable=False


temp_model=tf.keras.models.Sequential()
temp_model.add(base_model)
temp_model.summary()


## ResNet50V2


In [ ]:
import tensorflow as tf
base_model=tf.keras.applications.ResNet50V2(
    include_top=False,
    weights="imagenet",
    input_shape=(75,75,3)
)

# count=0
for layer in bas_model.layers[:148]:
    layer.trainable=False


temp_model=tf.keras.models.Sequential()
temp_model.add(bas_model)
temp_model.summary()

## ResNet101V2

In [ ]:
import tensorflow as tf
base_model=tf.keras.applications.ResNet101V2(
    include_top=False,
    weights="imagenet",
    input_shape=(75,75,3)
)


for layer in base_model.layers[:292]:
    layer.trainable=False


temp_model=tf.keras.models.Sequential()
temp_model.add(base_model)
temp_model.summary()

## InceptionResNetV2

In [ ]:

import tensorflow as tf
base_model=tf.keras.applications.InceptionResNetV2(
    include_top=False,
    weights="imagenet",
    input_shape=(75,75,3)
)


for layer in base_model.layers[:553]:
    layer.trainable=False


temp_model=tf.keras.models.Sequential()
temp_model.add(base_model)
temp_model.summary()

## Xception

In [ ]:
base_model=tf.keras.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_shape=(75,75,3)
)


for layer in base_model.layers[:72]:
    layer.trainable=False


temp_model=tf.keras.models.Sequential()
temp_model.add(base_model)
temp_model.summary()

## FeedForward Network Classifier

In [ ]:
from keras.models import Sequential

model=Sequential()
model.add(base_model)
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(1024,activation='relu'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(512,activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(256,activation='relu'))
model.add(tf.keras.layers.Dense(len(labels), activation='softmax'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.004)

model.compile(loss = 'categorical_crossentropy',
                 optimizer = optimizer,
                  metrics =['Accuracy','AUC','Precision','Recall'])
print(model.summary())


In [ ]:
checkpoint_path = "model_checkpoint_resnet152v2.keras"
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    monitor="val_accuracy",
    mode="max",
    save_best_only=True,
    verbose=1
)

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs,
    callbacks=[checkpoint]
)

In [ ]:
y_true = []
for _, y in test_dataset:
    y_true.append(y.numpy())

y_pred = []
for x, _ in test_dataset:
    y_pred.append(np.argmax(model.predict(x), axis=1))

y_true = np.concatenate([y for _, y in test_dataset], axis=0)
y_pred = np.concatenate([np.argmax(model.predict(x), axis=1) for x, _ in test_dataset], axis=0)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

precision = precision_score(np.argmax(y_true, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_true, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_true, axis=1), y_pred, average='weighted')
accuracy = accuracy_score(np.argmax(y_true, axis=1), y_pred)

print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")
print(f"Accuracy:  {accuracy:.4f}")

In [ ]:
print("\nClassification Report:\n")
print(classification_report(np.argmax(y_true, axis=1), y_pred, target_names=labels))

conf_matrix = confusion_matrix(np.argmax(y_true, axis=1), y_pred, labels=range(len(labels)))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
plt_epochs = list(range(50))

acc = history.history['Accuracy']
val_acc = history.history['val_Accuracy']

val_loss = history.history['val_loss']
loss = history.history['loss']

fig, ax = plt.subplots(1,3,figsize=(5, 0.1))

ax[0].plot(plt_epochs, acc, label='Train Accuracy')
ax[0].plot(plt_epochs, val_acc, label='Validation Accuracy')
ax[0].set_xlabel('Number of Epochs',weight='bold')
ax[0].set_ylabel('Accuracy',weight='bold')
ax[0].set_title('ResNet152V2 Model Accuracy',weight='bold')
ax[0].legend()
ax[0].legend(prop={'size': 8})

ax[1].plot(plt_epochs, loss, label='Train Loss')
ax[1].plot(plt_epochs, val_loss, label='Validation Loss')
ax[1].set_xlabel('Number of Epochs',weight='bold')
ax[1].set_ylabel('Loss',weight='bold')
ax[1].set_title('Model Loss',weight='bold')
ax[1].legend()
ax[1].legend(prop={'size': 8})

cm1 = conf_matrix


cm1= np.array(cm1)
label_names = labels
disp1 = ConfusionMatrixDisplay(confusion_matrix=cm1, display_labels=label_names)
disp1.plot(ax=ax[2],xticks_rotation='vertical',cmap='BuPu',text_kw={'fontsize': 8})
disp1.im_.colorbar.remove()
fig = disp1.ax_.get_figure()
fig.set_figwidth(9)
fig.set_figheight(2.3)
ax[2].set_xlabel('Predicted labels',weight='bold')
ax[2].set_ylabel('True labels',weight='bold')
ax[2].set_title('Confusion Matrix',weight='bold')
plt.subplots_adjust(hspace=0.5, wspace=0.5)
ax0 = ax[0].get_position()
ax1 = ax[1].get_position()
ax2 = ax[2].get_position()
# ax[2].set_xticks(weight = 'bold')
ax[0].set_position([ax0.x0,ax0.y0,.27,1])
ax[1].set_position([.46,ax1.y0,.27,1])
ax[2].set_position([.8,ax2.y0,.30,1])
plt.show()